In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import os

hrefs = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [13]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            if r.status_code == 508:
                print(r.status_code)
                print('error in link, retrying... ', url)
                time.sleep(20)
                continue
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "lxml")

    block = soup.find_all('h2',attrs = {"class":"entry-title"})
    if block is None:
        return

    for link in soup.find_all('h2',attrs = {"class":"entry-title"}):

        href = link.find('a').get('href')
        print(href)

        hrefs.append(href)
    

In [ ]:
max_worker = 10
hrefs = []

links = ["https://fiksyenshasha.com"]
for url in links:
    
    print(url)
    r = requests.get(url,headers = headers)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page')[1].replace('/','')) for a_ in pgno])
    print(max_page)
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}/page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()


In [7]:
len(hrefs)

7006

In [16]:
hrefs = list(set(hrefs))

In [18]:
with open('fiksyenshasha-links.json', 'a') as f:
    json.dump(hrefs, f)

In [6]:
with open('fiksyenshasha-links.json') as f:
    hrefs = json.load(f)

In [25]:
def process_url(x):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like
    comments = []
    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', class_="entry-title").text
        h = soup.find('div', attrs = {"class":"entry-content"})
        p = [p.text for p in h.find_all("p") if len(p.text.split(" ")) > 1 and 'fiksyenshasha.com/submit' not in p.text ]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None
    
    comment_list = soup.find('div',attrs = {"class":"comments-area"})
    if comment_list:
        c = comment_list.find_all('div',attrs = {"class":"comment-content"})
        for j in c:
            comments.extend([c_.text for c_ in j.find_all("p") if len(c_.text) > 1])
    
    data = {'url': x, 'headline': headline, 'content': p, 'comment': comments}
    return data

In [26]:
max_workers = 20

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('fiksyenshasha.jsonl', 'a',encoding='utf8') as f:
                json.dump(result, f,ensure_ascii=False)
                f.write('\n')

 32%|██████████████████████████████████▉                                                                           | 2224/7006 [02:29<07:46, 10.24it/s]

HTTPSConnectionPool(host='fiksyenshasha.com', port=443): Max retries exceeded with url: /saka-harimau/ (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_HANDSHAKE_FAILURE] sslv3 alert handshake failure (_ssl.c:1131)')))


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7006/7006 [09:00<00:00, 12.97it/s]


In [17]:
process_url("https://fiksyenshasha.com/pengalaman-di-asrama-ketika-pelajar-lain-bercuti/")

{'url': 'https://fiksyenshasha.com/pengalaman-di-asrama-ketika-pelajar-lain-bercuti/',
 'headline': 'Pengalaman di Asrama Ketika Pelajar Lain Bercuti',
 'content': ['asalamualaikum shahsha…',
  'saya adalah salah seorang pembaca setia page fiksyen Shasha.. memandangkan dah ramai share pengalaman masing-masing, saya pun ingin berkongsi kisah saya ni.. tidak la terlalu seram tapi ianya cukup membuatkan saya tidak dapat tidur lena malam tu..',
  'Kisah ni berlaku pada tahun 2013, ketika itu saya mendapat tawaran di sebuah kolej untuk kursus Safety lepasan diploma.. sebab dah saya bosan duduk dirumah, saya terima terima saja tawaran tersebut. lagipun kursus ni jangka pendek, 6 bulan sahaja. selepas membuat pendaftaran, kami ditempatkan diasrama bangunan lama yang sudah usang..',
  'Memandangkan kursus ini pendek maka tidak terdapar banyak bilik kosong yg disediakan oleh pihak berwajib, jadi saya tinggal di dorm bersama junior2 sijil dan diploma di tingkat 1.. keanehan pertama yg saya lihat